In [1]:
import requests # requests Library
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup # Library to handle html

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #create a link varible containing the html address
page = requests.get(link) # create variable that conatins the returned resulsts from request

In [3]:
page_soup = BeautifulSoup(page.text, 'html.parser') # use Beautiful Soup html parser to handle html code
t = page_soup.find('tbody') # Find First Table find_all tables also an option (Postal codes are included in the first table)
col = t.find_all('th') # Identify the Col Names to create empty Data Frame
rows = t.find_all('tr')[1:] # Identify Row Data to insert into empty Data Frame 

# loop through Html Columns, convert to strings and store in list t1
t1 = []
for i in range(len(col)):
    t1.append(str(col[i]).strip('<th>').strip('</').strip('\n'))

hoods = pd.DataFrame(columns = t1) # create data frame with colums headers using t1 list 

# loop through table rows contained in rows variable and add to the dataframe 
for i in range(len(rows)):
    hoods.loc[i, 'Postcode'] = rows[i].findAll(text=True)[1]
    hoods.loc[i, 'Borough'] = rows[i].findAll(text=True)[3]
    hoods.loc[i, 'Neighbourhood'] = str(rows[i].findAll(text=True)[5]).strip('\n')

print("DataFrame Shape:", hoods.shape, "Rows:", hoods.shape[0], "Columns:", hoods.shape[1]) # inspect dataframe shape
hoods.head() # review dataframe sample 

DataFrame Shape: (289, 3) Rows: 289 Columns: 3


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
df1 = hoods.drop(hoods[hoods.Borough == 'Not assigned'].index) # drop all row's where Borough is equal to "Not assigned" and create a new dataframe
df1 = df1.reset_index(drop=True) # reset new dataframe index  
print("DataFrame Shape:", df1.shape, "Rows:", df1.shape[0], "Columns:", df1.shape[1] ) # inspect new dataframe shape (it should contain less rows)
df1.head() # display a 10 row sample of the new dataframe (Showing no Boroughs = "Not assigned")

DataFrame Shape: (212, 3) Rows: 212 Columns: 3


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [5]:
# Iterate over the Values in the "Neighborhood" column and update None or Not Assigned values with the Borough name
for i in range(len(df1)):
    if df1['Neighbourhood'][i] == "Not assigned":
        df1['Neighbourhood'][i] = df1['Borough'][i]
    else:
        pass
print("DataFrame Shape:", df1.shape, "Rows:", df1.shape[0], "Columns:", df1.shape[1] ) # inspect dataframe shape (it should not change)
df1.head() # display a sample of the updated dataframe (all Niehborhoods now showing a value)

DataFrame Shape: (212, 3) Rows: 212 Columns: 3


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [6]:
boro = {k: g["Borough"].tolist() for k,g in df1.groupby("Postcode")} # Create dictionary with Postal code as the Key, and all corresponding Borough's as a list
hood = {k: g["Neighbourhood"].tolist() for k,g in df1.groupby("Postcode")} # Create a dictionary with Postal code as the Key, and all corresponding Neiborhood's as a list
t2 = pd.DataFrame(list(boro.keys()), columns = ['PostCode'])
for i, row in t2.iterrows():
    t2.loc[i,'Borough'] = list(set(list(boro.values())[i]))[0]    
    t2.loc[i,'Neighbourhood'] = ', '.join(list(set(list(hood.values())[i])))

print("DataFrame Shape:", t2.shape, "Rows:", t2.shape[0], "Columns:", t2.shape[1] ) # inspect dataframe shape (it should not change)
t2.head(10)

DataFrame Shape: (103, 3) Rows: 103 Columns: 3


,PostCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


In [7]:
t2[(t2['PostCode'] == 'M5A')] # test results by verifying 'M5A' Borough and Neighbourhood

,PostCode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"
